In [32]:
import numpy as np
import nlopt
import pandas as pd
from math import log, exp
from multiprocessing import Pool, cpu_count
import time
import matplotlib.pyplot as plt
%matplotlib inline
import gc

In [4]:
def rho(rbp,g,L,d):
    r = rbp*(d+2*(g/rbp)*L*(1 - exp(-d/L)))
    return(r)

In [19]:
def calc_single_dist_rho_only(x,r):
    d,h0,h1,h2,th = x
    r = r*(d)
    calcH0 =(18+13*r + r**2 + 36*th + 22*th**2 + 4*th**3 + r*(6*th+th**2))/((1+th)*(18+13*r+r**2+54*th + 40*th**2+8*th**3+r*(r*th+19*th+6*th**2)))
    calcH2 = (th**2*(36+14*r+r**2+36*th+6*th*r+8*th**2))/((1+th)*(18+13*r+r**2+54*th + 40*th**2+8*th**3+r*(r*th+19*th+6*th**2)))
    calcH1 = 1 - calcH0 - calcH2
    
    val = h0*log(calcH0) + h1*log(calcH1)+h2*log(calcH2)
    return(val)

In [20]:
def single_chr_obj_fun(r,arr):
    rbp = r
    test = np.apply_along_axis(lambda x: calc_single_dist_rho_only(x,rbp),1,arr)
    obfun = np.sum(test)
    #test = [calc_single_dist(x,rbp,g,L) for x in arr]
    #obfun = sum(test)
    return(obfun)

In [21]:
def across_chr_obj_fun(x,dfArrList,numChrs):
    
    rList = x[0:numChrs]
    
    obFun = sum([single_chr_obj_fun(rs,arr) for (rs,arr) in list(zip(rList,dfArrList))])
    return(obFun)

In [23]:
def combine_across_individuals(thisDF):
    test = thisDF
    numSamples = len(test['name'].unique())
    test = test.groupby(['distances'],as_index=False).sum()
    test['H'] = test['H']/numSamples
    return(test.iloc[:,[0,1,2,3,7]].to_numpy())
    #return(test.iloc[:,[0,4,5,6,11]].to_numpy())
    #return( test.iloc[:,[0,4,5,6,11]].to_numpy() )

In [24]:
def find_max_like_group_ind(numberChromosomes):
    autoNames = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8',
       'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19']
    dfListAuto = [ df.loc[(df['chrom']== x) & (df['distances']>=100)] for x in autoNames[0:numberChromosomes]]
    dfListAuto = [combine_across_individuals(x) for x in dfListAuto]

    lowerBounds = [0.0001 for x in range(numberChromosomes)]
    upperBounds = [0.1 for x in range(numberChromosomes)]
    startPoints = [0.001 for x in range(numberChromosomes)]


    opt=nlopt.opt(nlopt.LN_NELDERMEAD,numberChromosomes)

    opt.set_lower_bounds(lowerBounds)
    opt.set_upper_bounds(upperBounds)

    opt.set_max_objective(lambda x,grad: across_chr_obj_fun(x,dfListAuto,numberChromosomes))

    res = opt.optimize(startPoints)
    return(res)

In [16]:
df = pd.read_csv('~/Dropbox/projects/gene_conversion/mathematicaInputMice_unbinned.tsv',delimiter='\t')

In [17]:
df

,name,chrom,distances,no_het_bins,one_het_bins,two_het_bins,H0Prop,H1Prop,H2Prop,H
0,H12,chr1,1,62864864,844714.0,13604,0.986531,0.013256,0.000213,0.006841
1,H12,chr1,2,62850711,854466.0,8706,0.986452,0.013411,0.000137,0.006842
2,H12,chr1,3,62840557,856190.0,7807,0.986437,0.013440,0.000123,0.006843
3,H12,chr1,4,62831402,855898.0,7907,0.986438,0.013437,0.000124,0.006843
4,H12,chr1,5,62821707,856675.0,7467,0.986431,0.013452,0.000117,0.006843
...,...,...,...,...,...,...,...,...,...,...
199995,H36,chrX,996,33330821,249597.0,673,0.992547,0.007433,0.000020,0.003736
199996,H36,chrX,997,33326448,249349.0,779,0.992551,0.007426,0.000023,0.003736
199997,H36,chrX,998,33321896,249468.0,706,0.992548,0.007431,0.000021,0.003736
199998,H36,chrX,999,33317379,249504.0,668,0.992547,0.007433,0.000020,0.003736


In [26]:
coEstimatesAutosomes = find_max_like_group_ind(19)
coEstimatesAutosomes

array([0.0024282 , 0.00330779, 0.00340688, 0.00255148, 0.00330316,
       0.00285753, 0.00278443, 0.00404483, 0.00329535, 0.00371671,
       0.00294055, 0.00332221, 0.00386993, 0.0036916 , 0.00247676,
       0.00353591, 0.00306366, 0.00324592, 0.00486269])